<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Notebook_Tutorial_PySpark_Riesgo_Crediticio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía práctica: Clasificación de riesgo crediticio con PySpark y Spark ML

Este cuaderno tiene como objetivo guiar al estudiante paso a paso en la construcción de un pipeline de aprendizaje automático utilizando PySpark, para predecir el riesgo de incumplimiento de pago por parte de clientes de una entidad financiera. Se utiliza el conjunto de datos real 'Default of Credit Card Clients Dataset' disponible en Kaggle.

## Paso 1: Instalar PySpark (solo si está en Google Colab)

In [1]:
!pip install pyspark

## Paso 2: Importar librerías necesarias

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

## Paso 3: Crear una sesión de Spark

In [3]:
spark = SparkSession.builder.appName("TutorialRiesgoCrediticio").getOrCreate()

## Paso 4: Cargar el archivo de datos

Asegúrese de haber descargado y subido el archivo `UCI_Credit_Card.csv` desde Kaggle antes de ejecutar esta celda.

In [5]:
df = spark.read.csv("UCI_Credit_Card.csv", header=True, inferSchema=True)
df = df.withColumnRenamed("default.payment.next.month", "label")
df.select("LIMIT_BAL", "AGE", "EDUCATION", "MARRIAGE", "SEX", "label").show(5)

+---------+---+---------+--------+---+-----+
|LIMIT_BAL|AGE|EDUCATION|MARRIAGE|SEX|label|
+---------+---+---------+--------+---+-----+
|  20000.0| 24|        2|       1|  2|    1|
| 120000.0| 26|        2|       2|  2|    1|
|  90000.0| 34|        2|       2|  2|    0|
|  50000.0| 37|        2|       1|  2|    0|
|  50000.0| 57|        2|       1|  1|    0|
+---------+---+---------+--------+---+-----+
only showing top 5 rows



## Paso 5: Transformar variables categóricas
Convertimos las variables EDUCATION, MARRIAGE y SEX a índices numéricos para que puedan ser utilizadas por el modelo.

In [6]:
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_idx")
    for col in ["EDUCATION", "MARRIAGE", "SEX"]
]

## Paso 6: Combinar variables en un vector de características
Utilizamos `VectorAssembler` para agrupar las variables predictoras.

In [7]:
assembler = VectorAssembler(
    inputCols=["LIMIT_BAL", "AGE"] + [col+"_idx" for col in ["EDUCATION", "MARRIAGE", "SEX"]],
    outputCol="features_raw")

## Paso 7: Escalar las características
Para mejorar la estabilidad numérica del modelo, escalamos las características con `StandardScaler`.

In [8]:
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

## Paso 8: Definir modelo y pipeline

In [9]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
pipeline = Pipeline(stages=indexers + [assembler, scaler, lr])

## Paso 9: Entrenar el modelo y hacer predicciones

In [10]:
model = pipeline.fit(df)
predictions = model.transform(df)
predictions.select("label", "prediction", "probability").show(5)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       0.0|[0.69400241111226...|
|    1|       0.0|[0.78484208412147...|
|    0|       0.0|[0.75881277160846...|
|    0|       0.0|[0.69929370572321...|
|    0|       0.0|[0.63602236063087...|
+-----+----------+--------------------+
only showing top 5 rows



## Paso 10: Evaluar el modelo
Usamos el área bajo la curva ROC para evaluar la capacidad predictiva del modelo.

In [11]:
evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
print(f"Área bajo la curva (AUC): {auc:.4f}")

Área bajo la curva (AUC): 0.6223


## Conclusión
Este ejercicio muestra cómo construir un flujo completo de aprendizaje automático escalable utilizando PySpark y Spark ML. Se ha implementado la predicción de riesgo crediticio sobre un conjunto de datos real, incluyendo el preprocesamiento, ensamblaje, escalado, modelado y evaluación del rendimiento.